Function to read f_27 file

In [24]:
def leer_opta_f27(xml_file):
    ''''
    Function to read F27 Opta Pass Matrix file

    Input:
    OPTA F27 xml file


    '''

    import xml.etree.ElementTree as ET
    import pandas as pd

    file = ET.parse(xml_file)

    namelist= []
    xlist =[]
    ylist = []
    passer = []
    receiver = []
    passeslist = []
    poslist = []
    pass_lost_list = []

    for node in file.getroot():
        for players in node:
            name = node.attrib.get("player_name")
            x = float(node.attrib.get("x"))
            y = float(node.attrib.get("y"))
            pos = node.attrib.get("position")
            pass_lost = int(node.attrib.get("pass_lost"))
            namelist.append(name)
            xlist.append(x)
            ylist.append(y)
            poslist.append(pos)
            pass_lost_list.append(pass_lost)


            passes = players.text
            name = players.attrib.get("player_name")
            passer.append(node.attrib.get("player_name"))
            receiver.append(name)
            passeslist.append(passes)

    
    df=pd.DataFrame(data = list(zip(passer,xlist,ylist,poslist,receiver, passeslist)),
                    columns = ["Passer","x","y","Position_Passer","Receiver","PassSuccess"])
    
    return df.sort_values(by=["Position_Passer",'Passer']).reset_index(drop=True)

In [25]:
test=leer_opta_f27('C:/Users/Felix/Desktop/Máster  BD Deporte/Módulo 7 - Proveedores de Datos Deportivos/f27/pass_matrix_23_2019_g1074821_t957.xml')
test

,Passer,x,y,Position_Passer,Receiver,PassSuccess
0,Dimitrios Siovas,39.64,75.41,Defender,Youssef En-Nesyri,2
1,Dimitrios Siovas,39.64,75.41,Defender,Recio,2
2,Dimitrios Siovas,39.64,75.41,Defender,Marc Navarro,1
3,Dimitrios Siovas,39.64,75.41,Defender,Rubén Pérez,3
4,Dimitrios Siovas,39.64,75.41,Defender,Rodrigo Tarín,6
...,...,...,...,...,...,...
94,José Arnáiz,67.93,59.35,Substitute,Óscar Rodríguez,1
95,José Arnáiz,67.93,59.35,Substitute,Dimitrios Siovas,1
96,Recio,51.26,43.61,Substitute,Martin Braithwaite,1
97,Recio,51.26,43.61,Substitute,Marc Navarro,1
